In [24]:
import json
import os
import django
import sys
sys.path.append('..')
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'SMARTROLL.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'SMARTROLL.settings')
django.setup()
from Manage.models import College,Term,Branch,Stream,Semester,Division,Batch,Subject,TimeTable,GPSCoordinates,Classroom,Schedule,Lecture,Link,PermanentSubject,SubjectChoices,ComplementrySubjects,SubjectGroups
from Manage.serializers import PermanentSubjectSerializer
from StakeHolders.serializers import StudentSerializer
from Session.models import Attendance,Session
from StakeHolders.models import NotificationSubscriptions,SuperAdmin,Admin,Teacher,Student,Profile
from django.contrib.auth import get_user_model
import datetime
import pandas as pd
import string
from django.db.models import Count, Q
import json

In [11]:
semester_obj = Semester.objects.get(slug='914150_1731255589')

In [18]:
subject_groups_qs = SubjectGroups.objects.filter(semester=semester_obj)

In [16]:
subject_groups =  {}
# subject_group_students = {}
total_hours = {}
for subject_group in subject_groups_qs:
    subjects = subject_group.subjects.all()
    subject_groups[tuple(subjects)] = subject_group.students.all()
    # subject_group_students[tuple(subjects)] = subject_group.students.all()
    for subject in subjects:
        if subject not in total_hours:
            total_hours[subject] = subject.subject_map.L + subject.subject_map.T + subject.subject_map.P
subject_groups

{(<Subject: Compiler Design | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>,
  <Subject: Mobile Computing and Wireless Communication | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>,
  <Subject: Artificial Intelligence | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>,
  <Subject: Big Data Analytics | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>,
  <Subject: Mobile Application Development | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>,
  <Subject: Information security | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>): <QuerySet [<Student: 220283107033 | 22csman033@ldce.ac.in>]>,
 (<Subject: Compiler Design | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>,
  <Subject: Cloud Computing | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>,
  <Subject: Artificial Intelligence | Semester - 7 | S

In [19]:
subject_groups =  {}
# subject_group_students = {}
total_hours = {}
for subject_group in subject_groups_qs:
    subjects = frozenset(subject_group.subjects.all())
    subject_groups[subjects] = subject_group.students.all()
    # subject_group_students[tuple(subjects)] = subject_group.students.all()
    for subject in subjects:
        if subject not in total_hours:
            total_hours[subject] = subject.subject_map.L + subject.subject_map.T + subject.subject_map.P

# Constraints
min_students = 0  # No minimum size for divisions
max_students = 90  # Maximum size per division

def allocate_groups_with_splitting(subject_groups, total_hours, min_students, max_students):
    # Step 1: Preprocess subject groups to split any group exceeding max_students
    preprocessed_groups = []
    for group, students in subject_groups.items():
        student_list = list(students)  # Convert to a list if necessary
        while len(student_list) > max_students:
            preprocessed_groups.append((group, student_list[:max_students]))
            student_list = student_list[max_students:]
        if len(student_list) > 0:
            preprocessed_groups.append((group, student_list))

    # Step 2: Initialize variables for the backtracking algorithm
    groups = [group for group, _ in preprocessed_groups]
    students = [students for _, students in preprocessed_groups]
    best_solution = {"divisions": None, "deviation": float("inf")}
    recursive_calls = 0

    def calculate_hours(division):
        """Calculate total hours per subject in a division."""
        hours = {subject: 0 for subject in total_hours}
        for group, _ in division:
            for subject in group:
                hours[subject] = total_hours[subject]
        return hours

    def calculate_deviation(hours):
        """Calculate the total deviation of hours from the initial requirement."""
        return sum(abs(hours[sub] - total_hours[sub]) for sub in total_hours)

    def backtrack(current_divisions, remaining_groups, remaining_students):
        nonlocal recursive_calls
        recursive_calls += 1

        # Base case: no groups left to allocate
        if not remaining_groups:
            total_hours_all = {subject: 0 for subject in total_hours}
            for division in current_divisions:
                hours = calculate_hours(division)
                for subject in total_hours_all:
                    total_hours_all[subject] += hours[subject]
            deviation = calculate_deviation(total_hours_all)
            if deviation < best_solution["deviation"]:
                best_solution["divisions"] = [div[:] for div in current_divisions]
                best_solution["deviation"] = deviation
            return

        # Recursive case: allocate the next group
        group = remaining_groups[0]
        group_students = remaining_students[0]
        count = len(group_students)

        # Try allocating the group (or part of it) to each division
        for i in range(len(current_divisions)):
            division = current_divisions[i]
            division_student_count = sum(len(students) for _, students in division)

            if division_student_count + count <= max_students:
                # Allocate the entire group to this division
                division.append((group, group_students))
                backtrack(current_divisions, remaining_groups[1:], remaining_students[1:])
                division.pop()
            else:
                # Allocate a part of the group to this division
                split_count = max_students - division_student_count
                if split_count > 0:
                    division.append((group, group_students[:split_count]))
                    backtrack(
                        current_divisions,
                        [group] + remaining_groups[1:],
                        [group_students[split_count:]] + remaining_students[1:],
                    )
                    division.pop()

        # Start a new division with this group
        if len(current_divisions) < len(groups):
            current_divisions.append([(group, group_students)])
            backtrack(current_divisions, remaining_groups[1:], remaining_students[1:])
            current_divisions.pop()

    # Step 3: Initialize backtracking
    backtrack([], groups, students)

    # Log statistics
    print(f"Total Recursive Calls: {recursive_calls}")

    # Calculate the final subject hours for each subject
    total_hours_all_divisions = {subject: 0 for subject in total_hours}
    for division in best_solution["divisions"]:
        division_hours = calculate_hours(division)
        for subject in total_hours_all_divisions:
            total_hours_all_divisions[subject] += division_hours[subject]

    # Return the best solution with subject hours included
    return best_solution, total_hours_all_divisions


# Run the revised algorithm
result, subject_hours = allocate_groups_with_splitting(subject_groups, total_hours, min_students, max_students)
print("\nOptimal Division Allocation:")
if result["divisions"]:
    for i, division in enumerate(result["divisions"], start=0):        
        division_students = sum(len(students) for _, students in division)
        print(f"Division {i}: {division}")
        # we can create the division object and add the students to it
        # division_obj,created = Division.objects.get_or_create(semester=semester_obj,division_name=string.ascii_uppercase[i])
        # for group in division:
        #     division_obj.students.add(*group[1])
        print(f"Total Students: {division_students}")
# Print total deviation
print("\nTotal Deviation:", result["deviation"])

# # Print the subject hours for each subject in the final solution
print("\nFinal Subject Hours Across All Divisions:")
hour_deviations = {}
for subject, hours in subject_hours.items():
    hour_deviations[subject] = [total_hours[subject],hours]
print(hour_deviations)


Total Recursive Calls: 10593

Optimal Division Allocation:
Division 0: [(frozenset({<Subject: Compiler Design | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>, <Subject: Mobile Computing and Wireless Communication | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>, <Subject: Artificial Intelligence | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>, <Subject: Information security | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>, <Subject: Big Data Analytics | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>, <Subject: Mobile Application Development | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>}), [<Student: 200280107141 | Anisio Alberto da Conceicao Jose Mandlate>, <Student: 200280107142 | Adilson Jossias A. Gafur>, <Student: 220283107003 | amansevak123@gmail.com>, <Student: 220283107011 | dipenmagdani@gmail.com>, <Student: 220283107019 | pa

In [21]:
# Generate short form
def generate_short_form(subject):
    if type(subject) == str:
        return subject
    words = subject.subject_map.subject_name.split()
    short_form = ''.join(word[0] for word in words if word[0].isalpha()).upper()
    return short_form

def create_batches(subject, students, max_batch_size=24, single_batch_threshold=34):
    batches = {}
    students_list = list(students)  # Convert set to sorted list
    count = len(students_list)
    subject_shortform = generate_short_form(subject)
    
    if count <= single_batch_threshold:
        # Single batch case
        batches[f"{subject_shortform}1"] = students_list
    else:
        # Multiple batch case
        num_batches = max(1, (count + max_batch_size - 1) // max_batch_size)
        # Try to minimize the number of batches
        while True:
            base_size = count // num_batches
            extra = count % num_batches
            
            if base_size <= max_batch_size:
                break
            num_batches += 1
        
        # Distribute students across batches
        batch_sizes = [base_size + 1 if i < extra else base_size for i in range(num_batches)]
        start_index = 0
        
        for i, size in enumerate(batch_sizes, start=1):
            end_index = start_index + size
            batches[f"{subject_shortform}{i}"] = students_list[start_index:end_index]
            start_index = end_index

    return batches

In [25]:
divisions = {}
if result["divisions"]:
    for i, division in enumerate(result["divisions"]):
        division_name=string.ascii_uppercase[i]
        divisions[division_name] = {}
        subject_to_student_map = {division_name: set()}  # Initialize with 'common' key
        student_count_for_division = sum(len(students) for _, students in division)

        for group, students in division:
            student_set = set(students)
            subject_to_student_map[division_name].update(student_set)  # Add students to 'common'

            if len(division) != 1:
                for subject in group:
                    if subject.is_elective:
                        subject_to_student_map.setdefault(subject, set()).update(student_set)                
        # Now we can make batches        
        divisions[division_name]['total_batches'] = []
        divisions[division_name]['batches'] = []
        # make common batches for the divisions
        common_students = set()
        # if subject is elective them made serpate batches
        for subject,students in subject_to_student_map.items():            
            if  type(subject)!=str and not subject.is_elective:
                common_students.update(set(students))     
                continue          
            batches = create_batches(subject,students)
            for batch in batches:
                divisions[division_name]['total_batches'].append(batch)
                divisions[division_name]['batches'].append({'batch_name':batch,'student_count':len(batches[batch]),'students':batches[batch]})

print(divisions)

{'A': {'total_batches': ['A1', 'A2', 'A3', 'A4', 'MCAWC1', 'BDA1', 'MAD1', 'CC1', 'CC2', 'CC3', 'ML1', 'ML2', 'ML3', 'DF1', 'DF2', 'DF3'], 'batches': [{'batch_name': 'A1', 'student_count': 22, 'students': [<Student: 210280107077 | Maurya Ritik Rameshkumar>, <Student: 210280107081 | Henil Gajera>, <Student: 210280107100 | Punit Prajapati>, <Student: 210280107102 | Dhaduk Dhruv Ashokbhai>, <Student: 210280107103 | Chaudhari Tusharbhai Dineshbhai>, <Student: 210280107104 | Patel Kunj Rajeshkumar>, <Student: 210280107105 | Majevadiya Purvang Niteshbhai>, <Student: 210280107107 | Jignesh Bambhava>, <Student: 210280107108 | Lunagariya Disha Kishorbhai>, <Student: 210280107109 | Chavda Kajal Bhagvanbhai>, <Student: 210280107110 | CHIKHALIYA HIMANSHU KAILASHBHAI>, <Student: 210280107111 | JIGARSING NATVARJI THAKOR>, <Student: 210280107112 | Harsh Mehta>, <Student: 210280107113 | SANJAY DHUNDHALVA>, <Student: 210280107114 | Gojiya Krishna Nathabhai>, <Student: 210280107115 | Patel Dev Anilkumar

In [14]:
for batch in range(1,90,23):
    print(batch)

1
24
47
70
